# ASL Project

In [19]:
#Import libraries
import numpy as np
import random

# Imports to view data
import cv2
import pandas as pd
from glob import glob

# Visualization
from keras.utils import print_summary
from matplotlib import pyplot as plt

#ML libraries
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

In [20]:
#Directory paths
TRAIN_DIR = "../Dataset/asl_alphabet_train/asl_alphabet_train"
TEST_DIR = "../Dataset/asl_alphabet_test"
MODEL_DIR = './Model'
MODEL_PATH = MODEL_DIR+"/Model1-xception-2.h5"
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight1-xception-2.h5"

In [21]:
# Set global variables
TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3) # add channel for RGB
CLASSES = 29
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64
LABELS_NAME = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q',
               'R','S','T','U','V','W','X','Y','Z','del','nothing','space']

## Data Augmentation

In [22]:
#Load Train dataset
train_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=VALIDATION_SPLIT
)

validation_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    validation_split=VALIDATION_SPLIT
)

train_generator = train_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, shuffle=True, subset="training")
val_generator = validation_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


## Model

In [23]:
#Define Model Xception Model
def Xception_model_build():
    from keras.applications.xception import Xception
    from keras.layers import Input

    input_tensor = Input(shape=TARGET_DIMS)
    model = Xception(input_tensor = input_tensor, weights=None, include_top=True, classes= CLASSES)
    return model

In [24]:
from keras.models import Model
base_model = Xception_model_build()
x = base_model.output
#x = (Dense(CLASSES, activation='softmax'))(x)
model = Model(inputs=base_model.input, outputs=x)

#Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.summary()
model.save(MODEL_PATH)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 31, 31, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 31, 31, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 31, 31, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [28]:
#Checkpointer to save the best models
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*2 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*2 / BATCH_SIZE) )

model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=50, callbacks=[checkpointer], verbose=1)

Epoch 1/50
39/39 [==============================] - 270s 7s/step - loss: 3.5758 - acc: 0.0685 - val_loss: 15.1264 - val_acc: 0.0437

Epoch 00001: val_loss improved from inf to 15.12643, saving model to ./Model/Model_Weight1-xception-2.h5
Epoch 2/50
39/39 [==============================] - 272s 7s/step - loss: 3.5750 - acc: 0.0645 - val_loss: 15.3626 - val_acc: 0.0469

Epoch 00002: val_loss did not improve from 15.12643
Epoch 3/50
39/39 [==============================] - 272s 7s/step - loss: 3.4448 - acc: 0.0829 - val_loss: 15.7312 - val_acc: 0.0219

Epoch 00003: val_loss did not improve from 15.12643
Epoch 4/50
39/39 [==============================] - 273s 7s/step - loss: 3.3215 - acc: 0.0958 - val_loss: 15.7655 - val_acc: 0.0219

Epoch 00004: val_loss did not improve from 15.12643
Epoch 5/50
39/39 [==============================] - 274s 7s/step - loss: 3.2480 - acc: 0.0942 - val_loss: 15.7151 - val_acc: 0.0250

Epoch 00005: val_loss did not improve from 15.12643
Epoch 6/50
39/39 [====

39/39 [==============================] - 299s 8s/step - loss: 0.5480 - acc: 0.8093 - val_loss: 1.3361 - val_acc: 0.6813

Epoch 00041: val_loss did not improve from 1.06259
Epoch 42/50
39/39 [==============================] - 291s 7s/step - loss: 0.5513 - acc: 0.8101 - val_loss: 1.5730 - val_acc: 0.5875

Epoch 00042: val_loss did not improve from 1.06259
Epoch 43/50
39/39 [==============================] - 282s 7s/step - loss: 0.5339 - acc: 0.8253 - val_loss: 0.8837 - val_acc: 0.7000

Epoch 00043: val_loss improved from 1.06259 to 0.88372, saving model to ./Model/Model_Weight1-xception-2.h5
Epoch 44/50
39/39 [==============================] - 278s 7s/step - loss: 0.5250 - acc: 0.8145 - val_loss: 0.9941 - val_acc: 0.7188

Epoch 00044: val_loss did not improve from 0.88372
Epoch 45/50
39/39 [==============================] - 283s 7s/step - loss: 0.5143 - acc: 0.8289 - val_loss: 1.1972 - val_acc: 0.7057

Epoch 00045: val_loss did not improve from 0.88372
Epoch 46/50
39/39 [================

In [29]:
#Predict on validation dataset
predictions = model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['D', 'P', 'space', 'del', 'A', 'H', 'nothing', 'R', 'M', 'M', 'K', 'del', 'R', 'J', 'D', 'H', 'E', 'W', 'A', 'B', 'M', 'nothing', 'K', 'O', 'del', 'J', 'Z', 'A', 'L', 'O', 'E', 'H', 'Q', 'V', 'I', 'X', 'E', 'E', 'O', 'del', 'L', 'E', 'V', 'S', 'E', 'J', 'A', 'D', 'P', 'R', 'R', 'Q', 'A', 'K', 'N', 'C', 'E', 'M', 'L', 'Z', 'Q', 'J', 'Y', 'S'] 64
0.8570610880851746 , 0.78125


In [30]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [31]:
#Predict
test_generator.reset()
predictions = model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'V', 'V', 'W', 'X', 'Y', 'Z', 'nothing', 'space'] 28


In [32]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(results)

                              Filename Predictions
0         asl_alphabet_test\A_test.jpg           A
1         asl_alphabet_test\B_test.jpg           B
2         asl_alphabet_test\C_test.jpg           C
3         asl_alphabet_test\D_test.jpg           D
4         asl_alphabet_test\E_test.jpg           E
5         asl_alphabet_test\F_test.jpg           F
6         asl_alphabet_test\G_test.jpg           G
7         asl_alphabet_test\H_test.jpg           H
8         asl_alphabet_test\I_test.jpg           I
9         asl_alphabet_test\J_test.jpg           J
10        asl_alphabet_test\K_test.jpg           K
11        asl_alphabet_test\L_test.jpg           L
12        asl_alphabet_test\M_test.jpg           M
13        asl_alphabet_test\N_test.jpg           N
14        asl_alphabet_test\O_test.jpg           O
15        asl_alphabet_test\P_test.jpg           P
16        asl_alphabet_test\Q_test.jpg           Q
17        asl_alphabet_test\R_test.jpg           R
18        asl_alphabet_test\S_t

In [33]:
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 96.42857142857143


In [ ]:
#https://medium.com/@arindambaidya168/https-medium-com-arindambaidya168-using-keras-imagedatagenerator-b94a87cdefad
#https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720
#

In [8]:
#Define Model VGG16 Model with pretrained weight
def Xception_model_build_weight():
    from keras.applications.xception import Xception
    from keras.layers import Input

    input_tensor = Input(shape=TARGET_DIMS)
    model = Xception(input_tensor = input_tensor, weights='imagenet', include_top=False)
    return model

from keras.models import Model
weight_base_model = Xception_model_build_weight()
x1 = weight_base_model.output
#Add the fully-connected layers 
x1 = Flatten(name='flatten')(x1)
x1 = Dense(4096, activation='relu', name='fc1')(x1)
x1 = Dense(4096, activation='relu', name='fc2')(x1)
x1 = Dense(CLASSES, activation='softmax', name='predictions')(x1)
weight_model = Model(inputs=weight_base_model.input, outputs=x1)

#Compile the model
weight_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

weight_model.summary()
MODEL_PATH = MODEL_DIR+"/Model1-xception-withweight.h5"
weight_model.save(MODEL_PATH)

83689472/83683744 [==============================] - 566s 7us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 31, 31, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 31, 31, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 31, 31, 32)   0           block1_conv1_bn[0][0]            
__________________________________________

In [14]:
#Checkpointer to save the best models
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight1-xception-withweight.h5"
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*4 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*4 / BATCH_SIZE) )

weight_model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=10, callbacks=[checkpointer], verbose=1)

Epoch 1/10
77/77 [==============================] - 636s 8s/step - loss: 3.3678 - acc: 0.0363 - val_loss: 3.3692 - val_acc: 0.0451

Epoch 00001: val_loss improved from inf to 3.36922, saving model to ./Model/Model_Weight1-xception-withweight.h5
Epoch 2/10
77/77 [==============================] - 643s 8s/step - loss: 3.3676 - acc: 0.0351 - val_loss: 3.3671 - val_acc: 0.0434

Epoch 00002: val_loss improved from 3.36922 to 3.36706, saving model to ./Model/Model_Weight1-xception-withweight.h5
Epoch 3/10
77/77 [==============================] - 607s 8s/step - loss: 3.3676 - acc: 0.0327 - val_loss: 3.3672 - val_acc: 0.0312

Epoch 00003: val_loss did not improve from 3.36706
Epoch 4/10
77/77 [==============================] - 600s 8s/step - loss: 3.3676 - acc: 0.0367 - val_loss: 3.3678 - val_acc: 0.0399

Epoch 00004: val_loss did not improve from 3.36706
Epoch 5/10
77/77 [==============================] - 610s 8s/step - loss: 3.3677 - acc: 0.0345 - val_loss: 3.3664 - val_acc: 0.0503

Epoch 00

In [15]:
#Predict on validation dataset
predictions = weight_model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = weight_model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E'] 64
3.3691654205322266 , 0.015625


In [16]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [17]:
#Predict
test_generator.reset()
predictions = weight_model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E'] 28


In [18]:
filenames=test_generator.filenames
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 3.571428571428571
